In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re
path = 'a2_data.xlsx'

### Data preprocess 

#### Mange model-company table

Dimension 'Release Date' showed unreadable data, we convert it into the format 'Year-Month-Day'.

(3162, 16)


,Model,Release Date,Release Year,Model ID,RAM Capacity (Mb),Storage (Mb),CPU Clock (MHz),Display Diagonal (in),Display Width(px),Display Length(px),Width (mm),Length (mm),Depth (mm),Volume (cubic cm),Mass (grams),Pixel Density (per inch)
0,Psion Organiser II LZ 64,1989-01-01,1989,1,0.000000,0.000000,0.000000,0.050360,0.000000,0.000000,0.369714,0.118852,0.146889,0.059332,0.044181,0.000000
1,Hewlett-Packard 95LX (HP Jaguar),1991-04-01,1991,2,0.000214,0.000015,0.002230,0.223022,0.056911,0.047619,0.416098,0.147541,0.127646,0.065799,0.053879,0.030793
2,Psion Series 3,1991-06-01,1991,3,0.000092,0.000015,0.001895,0.187050,0.056911,0.023810,0.432469,0.147541,0.101988,0.056634,0.044181,0.035336
3,Hewlett-Packard 95LX 1MB RAM (HP Jaguar),1992-01-01,1992,4,0.000458,0.000015,0.002230,0.223022,0.056911,0.047619,0.416098,0.147541,0.127646,0.065799,0.053879,0.030793
4,Psion Series 3a,1993-03-01,1993,5,0.000214,0.000015,0.003386,0.258993,0.154472,0.063492,0.432469,0.147541,0.101988,0.056634,0.046336,0.129985


 Find duplicate model and delete it.

In [78]:
# analyse duplicates
df_product_copy = df_product.copy()
df_product_copy['Model'] = df_product_copy['Model'].apply(lambda x: re.sub('[^A-Za-z0-9]+', '',x).lower())
print(len(df_product_copy))
print(len(set(list(df_product_copy['Model']))))

3162
3155


Find duplicated data and get their model id. Kept one row of record.

In [79]:
# find duplicate model id
duplicates1 = []
for index,value in dict(Counter(list(df_product_copy['Model']))).items():
    if value!=1:
        duplicates1.append(list(df_product_copy[df_product_copy['Model']==index]['Model ID']))
#         print(df_model_company_test[df_model_company_test['Model']==index]['Model ID'])
delete_model_id1 = [row[0] for row in duplicates1]
delete_model_id1

[1014, 1419, 1462, 1974, 1977, 2422, 2879]

In [80]:
# for num in delete_model_id1:
df_product=df_product[~df_product['Model ID'].isin(delete_model_id1)]
df_product = df_product.dropna()
print(df_product.shape)
df_product.head()

(3155, 16)


,Model,Release Date,Release Year,Model ID,RAM Capacity (Mb),Storage (Mb),CPU Clock (MHz),Display Diagonal (in),Display Width(px),Display Length(px),Width (mm),Length (mm),Depth (mm),Volume (cubic cm),Mass (grams),Pixel Density (per inch)
0,Psion Organiser II LZ 64,1989-01-01,1989,1,0.000000,0.000000,0.000000,0.050360,0.000000,0.000000,0.369714,0.118852,0.146889,0.059332,0.044181,0.000000
1,Hewlett-Packard 95LX (HP Jaguar),1991-04-01,1991,2,0.000214,0.000015,0.002230,0.223022,0.056911,0.047619,0.416098,0.147541,0.127646,0.065799,0.053879,0.030793
2,Psion Series 3,1991-06-01,1991,3,0.000092,0.000015,0.001895,0.187050,0.056911,0.023810,0.432469,0.147541,0.101988,0.056634,0.044181,0.035336
3,Hewlett-Packard 95LX 1MB RAM (HP Jaguar),1992-01-01,1992,4,0.000458,0.000015,0.002230,0.223022,0.056911,0.047619,0.416098,0.147541,0.127646,0.065799,0.053879,0.030793
4,Psion Series 3a,1993-03-01,1993,5,0.000214,0.000015,0.003386,0.258993,0.154472,0.063492,0.432469,0.147541,0.101988,0.056634,0.046336,0.129985


#### Mange company id

After reading this table, select the desired columns.

In [50]:
# show Company ID
# select Company_real and drop duplicates to assure oe id to one company
df_company = pd.read_excel(path,sheet_name='Company ID').dropna()[['Company_real','New_ID']]
df_company['New_ID'] = df_company['New_ID'].apply(lambda x: int(x))
df_company.columns = ['Company_real', 'Company_ID']
df_company['Company_ID'] = df_company['Company_ID'].apply(lambda x: int(x))

df_company = df_company.drop_duplicates()
print(df_company.shape)
print(len(set(df_company['Company_ID'])))
print(min(df_company['Company_ID']),max(df_company['Company_ID']))
df_company.head()

(156, 2)
156
1 156


,Company_real,Company_ID
1,United Internet,141
3,3Com,1
5,AboCom Systems,2
7,Acer,3
9,"Shenzhen Ainol Electronics Co., Ltd",116


#### Merge two tables

In [53]:
da_m = pd.merge(df_model_company, df_company, on='Company_ID')
print(da_m.columns)
da_m

Index(['Model', 'Release Date', 'Release Year', 'Model ID', 'Company_ID',
       'Company_real'],
      dtype='object')


,Model,Release Date,Release Year,Model ID,Company_ID,Company_real
0,Casio Cassiopeia E-3000,2003-01-01,2003,260,26,Casio
1,Casio Cassiopeia IT-500 M30,2003-04-01,2003,282,26,Casio
2,Casio Cassiopeia IT-500 M30B,2003-04-01,2003,283,26,Casio
3,Casio Cassiopeia IT-500 M30CB,2003-04-01,2003,284,26,Casio
4,Casio Cassiopeia IT-500 M30CR,2003-04-01,2003,285,26,Casio
...,...,...,...,...,...,...
2832,Micromax A110 Canvas 2,2012-11-01,2012,3067,82,Micromax Informatics
2833,Micromax A90S Pixel,2012-11-01,2012,3068,82,Micromax Informatics
2834,Micromax A68 Smarty 4.0,2012-12-01,2012,3130,82,Micromax Informatics
2835,UMI X1,2012-11-01,2012,3100,138,UMI DIGI


### Q2: Which companies tried to lead the new markets with the new types of mobile device? 

In order to answer these questons, we should know the features in this table.

RAM Capacity: RAM is "random access memory". The speed and performance of the system are directly related to the RAM. That means the **larger** RAM, the **faster** speed of system will have.

Storage: Storage is an integral part of the computer, users can store and access data for a long time. **Larger** storage can store more files.

CPU Clock： Generally, a **higher** clock speed means a **faster** CPU. However, there are many other factors at play.

Display Diagonal (in)：The size of the screen is usually described by the length of the diagonal, that is, the distance between the diagonals, usually in inches. The **larger the bigger**.

Display Length(px),Width (mm),Length (mm),Depth (mm): length of display in pixel.  D is the diagonal (in centimeters or inches), W is the width (in pixels), and Depth is the height (in pixels).


Volume (cubic cm)： width-length-depth in mm

Mass (grams)：weight 

Pixel Density (per inch)：Pixel density is a calculation that returns the number of physical pixels per inch on a screen or display of a device. **High** density means **better** images, better fonts, smoother lines, or in other words, higher quality


In order to find companies that are trying to lead new markets through new mobile devices, one can analyze from two perspectives, one is the new technology leading the new market, and the other is the number of new products launched during the cycle.
First, analyze the number of new products launched by each company each year.

In [81]:
from collections import Counter
da_m_select = da_m[['Release Year','Company_real']]
year_list = list(set(list(da_m['Release Year'])))
for year in year_list:
    print(year, dictlist(da_m_select[da_m_select['Release Year']==year].Company_real))

2003 {'Casio': 6, 'Coxion': 1, 'HTC': 11, 'MiTAC International Corp': 11, 'Samsung': 5, 'Sony Corporation': 9, 'Zebra Technologies': 3, 'Kyocera Corporation': 1, 'Motorola': 3, 'E-TEN Information Systems Co., Ltd': 4, 'Nokia Corporation': 3, 'Toshiba Corporation': 4, 'Acer': 3, 'Orange UK': 2, 'Palm': 4, 'I-Mate': 2, 'NEC Corporationæ': 1, 'Panasonic': 1, 'Asus': 3, 'Hewlett-Packard': 8, 'NTT DOCOMO, Inc': 1, 'ViewSonic': 1, 'Hitachi, Ltd': 1, 'T-Mobile': 1, 'Dell': 3, 'Handspring Inc': 2, 'O2': 1, 'Fujitsu Ltd': 2, 'RoverComputers': 4, 'BlackBerry Limited': 1, 'Sendo': 1, 'Bsquare Corporation': 1, 'SiemensæAG': 1}
2004 {'Casio': 3, 'HTC': 24, 'MiTAC International Corp': 8, 'Samsung': 2, 'Sony Corporation': 2, 'Zebra Technologies': 6, 'Motorola': 3, 'E-TEN Information Systems Co., Ltd': 2, 'Nokia Corporation': 4, 'Toshiba Corporation': 2, 'Acer': 3, 'Orange UK': 3, 'Palm': 4, 'I-Mate': 5, 'NEC Corporationæ': 1, 'Panasonic': 3, 'Asus': 2, 'Hewlett-Packard': 9, 'ViewSonic': 1, 'T-Mobile'